# Build giant tables

In [1]:
import os, re
import numpy as np
import pandas as pd
import scipy.io
from scipy.stats import iqr
from scipy import interpolate
from ieeg.auth import Session
from iEEG_helper_functions import *

SPIKES_OUTPUT_DIR = "../../Data/spikes/devin_spikes/"

In [2]:
# Load HUP_implant_dates.xlsx
patients_df = pd.read_excel("../../Data/HUP_implant_dates.xlsx")
patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6
...,...,...,...,...,...,...
75,141,HUP141_phaseII,2017-05-24,07:15:00,2017-06-01 00:00:00,85.7
76,140,HUP140_phaseII_D01-D02,2017-05-10,07:15:00,2017-05-19 00:00:00,56.7
77,139,HUP139_phaseII,2017-04-26,07:15:00,2017-05-09 00:00:00,69.8
78,138,HUP138_phaseII,2017-04-12,07:15:00,2017-04-20 00:00:00,84.4


In [3]:
# Create a mapping between patient ids and the index of the patient in the patients_df dataframe
patient_hup_id_to_index = {}
for i, patient_id in enumerate(patients_df["hup_id"]):
    patient_hup_id_to_index[patient_id] = i

In [4]:
# Initialize an empty list to hold the data
# completed_hup_ids = [160, 172, 141, 145, 157, 161, 138, 142, 151, 171, 175, 187]
completed_hup_ids = [
    160,
    172,
    # 141, # not enough time before first seizure
    145,
    138,
    142,
    151,
    187,
    180,
    184,
    # 192, # incomplete data
    # 196, # not enough time before first seizure
    # 204, # not enough time before first seizure
    # 165, # incomplete data
    # 169, # not enough time after the last seizure
    173,
    # 150, # not enough time before first seizure
    # 154, # incomplete data
    # 158, # incomplete data
    # 207, # not enough time before first seizure
    223,
    # 192,  ## Monday, August 21, 2023 additions this line and below # incomplete data
    # 196, # not enough time before first seizure
    # 204, # not enough time before first seizure
    177,
    185,
    # 189, # not enough time before first seizure
    # 205, # not enough time before first seizure
    166,
    # 170, # not enough time before first seizure
    # 174, # not enough time before first seizure
]
# Sort completed_hup_ids in ascending order
completed_hup_ids.sort()

In [5]:
# Only keep the rows in patients_df that correspond to the completed_hup_ids
patients_df = patients_df[patients_df["hup_id"].isin(completed_hup_ids)]
# reset the index of patients_df
patients_df = patients_df.reset_index(drop=True)
patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
1,187,HUP187_phaseII,2019-03-11,07:15:00,2019-03-19 00:00:00,86.2
2,185,HUP185_phaseII,2019-01-21,07:15:00,2019-02-01 00:00:00,76.2
3,184,HUP184_phaseII,2019-01-14,07:15:00,2019-01-21 00:00:00,90.7
4,180,HUP180_phaseII,2018-11-05,07:15:00,2018-11-13 00:00:00,51.3
5,177,HUP177_phaseII,2018-08-22,07:15:00,2018-08-31 00:00:00,88.4
6,173,HUP173_phaseII,2018-07-02,07:15:00,2018-07-13 00:00:00,76.6
7,172,HUP172_phaseII,2018-06-16,07:15:00,2018-06-29 00:00:00,109.4
8,166,HUP166_phaseII,2018-03-19,07:15:00,2018-03-28 00:00:00,60.3
9,160,HUP160_phaseII,2018-01-17,07:15:00,2018-02-01 00:00:00,80.6


In [6]:
ieeg_offset_df = pd.read_excel("../../Data/ieeg_offset_new.xlsx")
ieeg_offset_df

,hup_id,ieeg_offset_1,ieeg_offset_2,ieeg_offset_3,ieeg_offset_4
0,225,136590,NaN,NaN,NaN
1,224,135178,NaN,NaN,NaN
2,223,118454,NaN,NaN,NaN
3,221,135123,NaN,NaN,NaN
4,219,119758,NaN,NaN,NaN
...,...,...,...,...,...
75,141,133760,NaN,NaN,NaN
76,140,126678,283535.0,NaN,NaN
77,139,133613,NaN,NaN,NaN
78,138,134989,NaN,NaN,NaN


In [7]:
all_med_names = []

for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id = row.hup_id

    # Load HUP_{patient_hup_id}.npy from ../../Data/medications
    aed_np_file = np.load(
        f"../../Data/medications/HUP_{patient_hup_id}.npy", allow_pickle=True
    )

    all_dose_curves_plot = aed_np_file[0]
    all_tHr_plot = aed_np_file[1]
    all_med_names_plot = aed_np_file[2]

    # Plot dose curves
    for med_name in all_med_names_plot:
        all_med_names.append(med_name)

all_med_names = np.unique(np.array(all_med_names, dtype=str))
all_med_names

array(['carbamazepine', 'clobazam', 'clonazepam', 'eslicarbazepine',
       'lacosamide', 'lamotrigine', 'levetiracetam', 'lorazepam',
       'oxcarbazepine', 'topiramate'], dtype='<U15')

In [8]:
# Load aed_ref_ranges.xlsx from ./data/
aed_ref_ranges_df = pd.read_excel("../../Data/aed_ref_ranges.xlsx")
# Lowercase Drug column
aed_ref_ranges_df["Drug"] = aed_ref_ranges_df["Drug"].str.lower()
# show unique units
print(aed_ref_ranges_df["Unit"].unique())
# mg/L and ug/mL are the same
# If Unit is ng/mL, convert to ug/mL
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Min"] = (
    aed_ref_ranges_df["Min"] / 1000
)
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Max"] = (
    aed_ref_ranges_df["Max"] / 1000
)
# Add a column that takes the average of Min and Max
aed_ref_ranges_df["Avg"] = (aed_ref_ranges_df["Min"] + aed_ref_ranges_df["Max"]) / 2
aed_ref_ranges_df

['mg/L' 'ug/mL' 'ng/mL']


,Drug,Min,Max,Unit,Avg
0,levetiracetam,12.00,46.00,mg/L,29.000
1,carbamazepine,4.00,10.00,mg/L,7.000
2,oxcarbazepine,3.00,35.00,ug/mL,19.000
3,clobazam,0.03,0.30,ng/mL,0.165
4,n-desmethylclobazam,0.30,3.00,ng/mL,1.650
5,topiramate,5.00,20.00,mg/L,12.500
6,valproic acid,50.00,125.00,ug/mL,87.500
7,lacosamide,1.00,10.00,ug/mL,5.500
8,felbamate,30.00,60.00,ug/mL,45.000
9,lamotrigine,2.50,15.00,mg/L,8.750


## Spikes stuff

In [9]:
with open("dma_ieeglogin.bin", "r") as f:
    session = Session("dma", f.read())
# Initialize a dataframe with hup_id, dataset_name, fs
patients_df = pd.DataFrame(columns=["hup_id", "dataset_name", "fs"])
for patient_hup_id in completed_hup_ids:
    dataset_name = f"HUP{patient_hup_id}_phaseII"
    dataset = session.open_dataset(dataset_name)
    all_channel_labels = np.array(dataset.get_channel_labels())
    channel_labels_to_download = all_channel_labels[
        electrode_selection(all_channel_labels)
    ]

    fs = int(dataset.get_time_series_details(channel_labels_to_download[0]).sample_rate)
    # Construct a row and add it to the dataframe
    row_df = pd.DataFrame(
        [{"hup_id": patient_hup_id, "dataset_name": dataset_name, "fs": fs}]
    )
    patients_df = pd.concat([patients_df, row_df], ignore_index=True)
# Sort the dataframe by hup_id
patients_df = patients_df.sort_values(by="hup_id")
# reset the index
patients_df = patients_df.reset_index(drop=True)
patients_df

,hup_id,dataset_name,fs
0,138,HUP138_phaseII,1024
1,142,HUP142_phaseII,512
2,145,HUP145_phaseII,512
3,151,HUP151_phaseII,512
4,160,HUP160_phaseII,1024
5,166,HUP166_phaseII,1024
6,172,HUP172_phaseII,512
7,173,HUP173_phaseII,256
8,177,HUP177_phaseII,512
9,180,HUP180_phaseII,512


In [10]:
all_spikes_dfs = []
all_fs = []

for index, row in patients_df.iterrows():
    patient_hup_id = row["hup_id"]
    fs = row["fs"]
    all_fs.append(fs)
    print(f"Processing HUP {patient_hup_id} with fs {fs}")

    ###############################
    # Construct spike_files_df
    ###############################

    # Initialize an empty list to hold the data
    data = []

    # Iterate through all files in the directory
    for filename in os.listdir(SPIKES_OUTPUT_DIR):
        # Check if the file ends with .npy
        if filename.endswith(".npy"):
            # Use regular expression to match the pattern and extract desired numbers
            match = re.match(r"HUP(\d+)_phaseII_(\d+).npy", filename)

            if match:
                current_patient_hup_id = int(match.group(1))
                if current_patient_hup_id != patient_hup_id:
                    continue
                interval_index = int(match.group(2))

                # Append the data to the list
                data.append(
                    {
                        "filename": filename,
                        "interval_index": interval_index,
                    }
                )

    # Convert the list of dictionaries to a pandas DataFrame
    spike_files_df = pd.DataFrame(data)
    # Sort the DataFrame by the interval index
    spike_files_df = spike_files_df.sort_values(by="interval_index")
    # Reset the index
    spike_files_df = spike_files_df.reset_index(drop=True)
    # Add a new column called "start_sample_index"
    spike_files_df["start_sample_index"] = (
        spike_files_df["interval_index"] * fs * 60 * 2
    )

    ###############################
    # Construct all_spikes_df
    ###############################
    # Initialize an empty list to store individual DataFrames
    dfs = []

    for index, row in spike_files_df.iterrows():
        filename = row["filename"]
        start_sample_index = row["start_sample_index"]

        # Load the data
        spike_data = np.load(os.path.join(SPIKES_OUTPUT_DIR, filename))

        # Adjust the start_sample_index
        spike_data[:, 0] += start_sample_index

        # Convert the modified spike_data to a DataFrame and append to the dfs list
        dfs.append(
            pd.DataFrame(
                spike_data,
                columns=[
                    "peak_index",
                    "channel_index",
                    "peak",
                    "left_point",
                    "right_point",
                    "slow_end",
                    "slow_max",
                    "rise_amp",
                    "decay_amp",
                    "slow_width",
                    "slow_amp",
                    "rise_slope",
                    "decay_slope",
                    "average_amp",
                    "linelen",
                ],
            )
        )

    # Concatenate all the individual DataFrames into a single DataFrame
    all_spikes_df = pd.concat(dfs, ignore_index=True)
    # Drop any rows with any NaN values
    all_spikes_df = all_spikes_df.dropna().reset_index(drop=True)
    # Make peak_index and channel_index integers
    all_spikes_df["peak_index"] = all_spikes_df["peak_index"].astype(int)
    all_spikes_df["channel_index"] = all_spikes_df["channel_index"].astype(int)

    ###############################
    # ISI
    ###############################
    # Calculate the inter-spike interval
    all_spikes_df["inter_spike_interval_samples"] = all_spikes_df["peak_index"].diff()

    # Drop the first row and reset index
    all_spikes_df = all_spikes_df.dropna().reset_index(drop=True)

    # Convert the inter_spike_interval_samples column to integer
    all_spikes_df["inter_spike_interval_samples"] = all_spikes_df[
        "inter_spike_interval_samples"
    ].astype(int)

    all_spikes_df["inter_spike_interval_sec"] = (
        all_spikes_df["inter_spike_interval_samples"] / fs
    ).astype(int)

    all_spikes_dfs.append(all_spikes_df)

Processing HUP 138 with fs 1024
Processing HUP 142 with fs 512
Processing HUP 145 with fs 512
Processing HUP 151 with fs 512
Processing HUP 160 with fs 1024
Processing HUP 166 with fs 1024
Processing HUP 172 with fs 512
Processing HUP 173 with fs 256
Processing HUP 177 with fs 512
Processing HUP 180 with fs 512
Processing HUP 184 with fs 512
Processing HUP 185 with fs 512
Processing HUP 187 with fs 512
Processing HUP 223 with fs 1024


In [11]:
thresholds = [4.6, 4, 4.3, 4, 5.5, 4.1, 4, 3, 4, 4, 4, 3.7, 4, 5]
thresholds = np.exp(thresholds)
thresholds, len(thresholds)

(array([ 99.48431564,  54.59815003,  73.6997937 ,  54.59815003,
        244.69193226,  60.3402876 ,  54.59815003,  20.08553692,
         54.59815003,  54.59815003,  54.59815003,  40.44730436,
         54.59815003, 148.4131591 ]),
 14)

In [12]:
for all_spikes_df, fs, hup_id, threshold in zip(
    all_spikes_dfs, all_fs, completed_hup_ids, thresholds
):
    print(f"HUP {hup_id} with fs {fs}")
    # Convert peak_index to second
    all_spikes_df["peak_second"] = all_spikes_df["peak_index"] // fs
    # Convert peak_index to minute
    all_spikes_df["peak_minute"] = all_spikes_df["peak_index"] / fs // 60
    # Convert peak_index to hour
    all_spikes_df["peak_hour"] = all_spikes_df["peak_index"] / fs // 3600

HUP 138 with fs 1024
HUP 142 with fs 512
HUP 145 with fs 512
HUP 151 with fs 512
HUP 160 with fs 1024
HUP 166 with fs 1024
HUP 172 with fs 512
HUP 173 with fs 256
HUP 177 with fs 512
HUP 180 with fs 512
HUP 184 with fs 512
HUP 185 with fs 512
HUP 187 with fs 512
HUP 223 with fs 1024


## Main loop

In [13]:
for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id = row.hup_id
    patient_idx = patient_hup_id_to_index[patient_hup_id]
    fs = row.fs
    all_spikes_df = all_spikes_dfs[i]
    threshold = thresholds[i]
    print(f"Processing HUP {patient_hup_id}")

    # Find the ieeg_offset_1 value for patient_hup_id in ieeg_offset_df and convert it into float
    ieeg_offset_seconds = float(
        ieeg_offset_df.loc[
            ieeg_offset_df["hup_id"] == patient_hup_id, "ieeg_offset_1"
        ].values[0]
    )
    print(f"ieeg_offset_seconds: {ieeg_offset_seconds}")
    ieeg_offset_minutes = ieeg_offset_seconds / 60

    ##############################################
    # MEDICATIONS
    ##############################################
    # Load HUP_{patient_hup_id}.npy from ../../Data/medications
    aed_np_file = np.load(
        f"../../Data/medications/HUP_{patient_hup_id}.npy", allow_pickle=True
    )

    all_dose_curves_plot = aed_np_file[0]
    all_tHr_plot = aed_np_file[1]
    all_med_names_plot = aed_np_file[2]

    # Construct the time axis
    emu_start_time_hrs = min([all_tHr_plot[i][0] for i in range(len(all_tHr_plot))])
    emu_end_time_hrs = all_tHr_plot[0][-1]
    max_length = max([len(all_tHr_plot[i]) for i in range(len(all_tHr_plot))])
    time_axis = np.linspace(emu_start_time_hrs, emu_end_time_hrs, max_length)

    first_emu_hr = time_axis[0]

    # Create a dataframe that will hold the dose curves for all patients
    hourly_patient_features_df = pd.DataFrame(columns=["emu_time"])
    hourly_patient_features_df["emu_time"] = time_axis

    for potential_med_name in all_med_names:
        hourly_patient_features_df[f"med_{potential_med_name}_raw"] = np.zeros(
            len(time_axis)
        )

    sum_array = []

    ##############################################
    # MEDICATIONS Normalize to 1
    ##############################################
    for med_idx, med_name in enumerate(all_med_names_plot):
        dose_times = all_tHr_plot[med_idx].flatten()
        dose = all_dose_curves_plot[med_idx].flatten()

        interp_func = interpolate.interp1d(
            dose_times, dose, bounds_error=False, fill_value=0
        )
        dose_interp = interp_func(time_axis)

        if med_name != "lorazepam":
            sum_array.append(dose_interp)

        hourly_patient_features_df[f"med_{med_name}_raw"] = dose_interp

    cumulative_dose_curve = np.sum(sum_array, axis=0)
    cumulative_dose_curve = cumulative_dose_curve / np.max(cumulative_dose_curve)

    assert len(cumulative_dose_curve) == len(
        time_axis
    ), "cumulative_dose_curve and time_axis should have the same length"

    hourly_patient_features_df["med_sum_no_lorazepam_raw"] = cumulative_dose_curve

    ##############################################
    # MEDICATIONS Normalize with DDD
    ##############################################
    for med_idx, med_name in enumerate(all_med_names_plot):
        dose_times = all_tHr_plot[med_idx].flatten()

        # Find Avg for medication med_name in aed_ref_ranges_df
        if med_name != "lorazepam":
            ref_range = float(
                aed_ref_ranges_df.loc[
                    aed_ref_ranges_df["Drug"] == med_name, "Avg"
                ].values[0]
            )
        else:
            ref_range = 1

        dose = all_dose_curves_plot[med_idx].flatten()
        dose = dose / ref_range

        interp_func = interpolate.interp1d(
            dose_times, dose, bounds_error=False, fill_value=0
        )
        dose_interp = interp_func(time_axis)

        if med_name != "lorazepam":
            sum_array.append(dose_interp)

        hourly_patient_features_df[f"med_{med_name}_raw"] = dose_interp

    cumulative_dose_curve = np.sum(sum_array, axis=0)

    assert len(cumulative_dose_curve) == len(
        time_axis
    ), "cumulative_dose_curve and time_axis should have the same length"

    hourly_patient_features_df["med_sum_no_lorazepam_ddd"] = cumulative_dose_curve

    ##############################################
    # Group by 2 minutes and compute mean
    ##############################################
    hourly_patient_features_df["emu_minute"] = (
        (hourly_patient_features_df["emu_time"] * 60).astype(int) // 2 * 2
    )
    hourly_patient_features_df = hourly_patient_features_df.groupby("emu_minute").mean()
    hourly_patient_features_df = hourly_patient_features_df.reset_index()
    hourly_patient_features_df = hourly_patient_features_df.drop(columns=["emu_time"])

    ##############################################
    # SEIZURE COUNT
    ##############################################
    seizure_times_sec = np.load(
        f"../../Data/seizures/source_mat/HUP_{patient_hup_id}.npy"
    )
    seizure_times_sec = seizure_times_sec + ieeg_offset_seconds

    # Convert seizure times from seconds to minutes
    seizure_times_min = seizure_times_sec / 60

    hourly_patient_features_df["had_seizure"] = np.zeros(
        len(hourly_patient_features_df), dtype=int
    )

    for sz_min in seizure_times_min[:, 0]:
        hourly_patient_features_df.loc[
            hourly_patient_features_df["emu_minute"] == int(sz_min) // 2 * 2,
            "had_seizure",
        ] += 1

    ##############################################
    # Time since last seizure
    ##############################################
    # Initialize the list and timer
    time_since_last_seizure = []
    timer = None

    # Loop through the dataframe and calculate the time since the last seizure
    for had_seizure in hourly_patient_features_df["had_seizure"]:
        if had_seizure == 1:
            timer = 0
        elif timer is not None:  # if there has been a seizure before
            timer += 2
        else:
            timer = None
        time_since_last_seizure.append(timer)

    # Add the list as a new column
    hourly_patient_features_df["time_since_last_seizure"] = time_since_last_seizure

    ##########################################
    # SYNCHRONY
    ##########################################

    # Determine the starting index for the synchrony data
    start_index = None
    for i, emu_min in enumerate(hourly_patient_features_df["emu_minute"]):
        if i < len(hourly_patient_features_df["emu_minute"]) - 1:
            next_emu_min = hourly_patient_features_df["emu_minute"].iloc[i + 1]
        else:
            next_emu_min = emu_min + 2

        if emu_min <= ieeg_offset_minutes < next_emu_min:
            start_index = i
            break

    if start_index is None:
        print("start_index is actually 0...")
        start_index = 0

    synchrony_np = np.load(
        f"../../Data/synchrony/all/broadband/HUP_{patient_hup_id}.npy"
    )

    # Initialize the synchrony column with NaNs
    hourly_patient_features_df[f"synchrony_broadband"] = np.nan

    # Insert synchrony values starting from the appropriate index
    end_index = min(start_index + len(synchrony_np), len(hourly_patient_features_df))
    hourly_patient_features_df.iloc[
        start_index:end_index,
        hourly_patient_features_df.columns.get_loc(f"synchrony_broadband"),
    ] = synchrony_np[: end_index - start_index]

    ##########################################
    # AD Ratio
    ##########################################
    mat_file = scipy.io.loadmat(
        f"../../../erinconr/projects/fc_toolbox/results/analysis/intermediate/HUP{patient_hup_id}.mat"
    )
    mat_file = mat_file["summ"][0][0]
    ad_ratio = mat_file[17]
    num_channels = mat_file[6].shape[0]
    assert ad_ratio.shape[0] == num_channels

    ad_ratio = np.nanmean(ad_ratio, axis=0)
    ad_ratio = (ad_ratio - np.nanmedian(ad_ratio)) / iqr(ad_ratio, nan_policy="omit")
    assert np.nansum(ad_ratio) != 0

    # Reshape ad_ratio to match the granularity of the dataframe
    reshaped_ad_ratio = np.repeat(ad_ratio, 5)

    # Initialize the ad_ratio column with NaNs
    hourly_patient_features_df["ad_ratio"] = np.nan

    # Insert reshaped_ad_ratio values starting from the appropriate index
    end_index_ad_ratio = min(
        start_index + len(reshaped_ad_ratio), len(hourly_patient_features_df)
    )
    hourly_patient_features_df.iloc[
        start_index:end_index_ad_ratio,
        hourly_patient_features_df.columns.get_loc("ad_ratio"),
    ] = reshaped_ad_ratio[: end_index_ad_ratio - start_index]

    ##########################################
    # EEG time
    ##########################################
    # Create the eeg_time column with NaN values
    hourly_patient_features_df["eeg_time"] = np.nan

    # Define the eeg_time values starting from start_index
    eeg_time_values = np.arange(
        0, (end_index_ad_ratio - start_index) * 2, 2
    )  # incrementing by 2 since the time is grouped by 2 minutes
    hourly_patient_features_df.iloc[
        start_index:end_index_ad_ratio,
        hourly_patient_features_df.columns.get_loc("eeg_time"),
    ] = eeg_time_values

    ##########################################
    # SPIKES
    ##########################################
    filtered_df = all_spikes_df[
        all_spikes_df.inter_spike_interval_samples < threshold
    ].copy()

    spike_rate_array = np.full_like(synchrony_np, np.nan)
    spike_isi_array = np.full_like(synchrony_np, np.nan)
    spike_fano_factor_array = np.full_like(synchrony_np, np.nan)
    spike_num_channels_array = np.full_like(synchrony_np, np.nan)

    for i in range(len(synchrony_np)):
        current_df = filtered_df[
            (filtered_df["peak_minute"] >= i * 2)
            & (filtered_df["peak_minute"] < i * 2 + 2)
        ]

        spike_rate_array[i] = current_df.shape[0]
        spike_isi_array[i] = current_df["inter_spike_interval_samples"].mean()

        # Compute Fano Factor = Variance(ISI) / Mean(ISI)
        mean_isi = current_df["inter_spike_interval_samples"].mean()
        if mean_isi != 0:
            spike_fano_factor_array[i] = (
                current_df["inter_spike_interval_samples"].var() / mean_isi
            )
        else:
            spike_fano_factor_array[i] = np.nan

        spike_num_channels_array[i] = current_df["channel_index"].nunique() // (
            current_df["channel_index"].max() + 1
        )

    # Normalize spike_rate_array to have a max of 1
    spike_rate_array = spike_rate_array / np.nanmax(spike_rate_array)

    # Add spike metrics to dataframe
    hourly_patient_features_df["spike_rate"] = np.nan
    hourly_patient_features_df["spike_isi"] = np.nan
    hourly_patient_features_df["spike_fano_factor"] = np.nan
    hourly_patient_features_df["spike_num_channels"] = np.nan

    hourly_patient_features_df.iloc[
        start_index:end_index,
        hourly_patient_features_df.columns.get_loc("spike_rate"),
    ] = spike_rate_array[: end_index - start_index]

    hourly_patient_features_df.iloc[
        start_index:end_index,
        hourly_patient_features_df.columns.get_loc("spike_isi"),
    ] = spike_isi_array[: end_index - start_index]

    hourly_patient_features_df.iloc[
        start_index:end_index,
        hourly_patient_features_df.columns.get_loc("spike_fano_factor"),
    ] = spike_fano_factor_array[: end_index - start_index]

    hourly_patient_features_df.iloc[
        start_index:end_index,
        hourly_patient_features_df.columns.get_loc("spike_num_channels"),
    ] = spike_num_channels_array[: end_index - start_index]

    ##############################################
    # SAVE TO CSV
    ##############################################

    hourly_patient_features_df.to_csv(
        f"../../Data/giant_new_tables/HUP_{patient_hup_id}.csv", index=False
    )

Processing HUP 138
ieeg_offset_seconds: 134989.0


/tmp/ipykernel_28100/1296569811.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 142
ieeg_offset_seconds: 127857.0


/tmp/ipykernel_28100/1296569811.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 145
ieeg_offset_seconds: 137942.0


...

/tmp/ipykernel_28100/1296569811.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 187
ieeg_offset_seconds: 121019.0


/tmp/ipykernel_28100/1296569811.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)


Processing HUP 223
ieeg_offset_seconds: 118454.0


/tmp/ipykernel_28100/1296569811.py:209: RuntimeWarning: Mean of empty slice
  ad_ratio = np.nanmean(ad_ratio, axis=0)
